In [42]:
import rsa
import multiprocessing

Reference:

rsa: https://stuvel.eu/python-rsa-doc/usage.html#signing-and-verification  

pycrypotodome: https://www.pycryptodome.org/en/latest/index.html

# environment

## install rsa

`pip install rsa`

The RSA in pycryptodome is difficult to use

## install pycryptodome

*Never install PyCrypto,use pycryptodome !*

1.a drop-in replacement for the old PyCrypto library. You install it with:`pip install pycryptodome`In this case, all modules are installed under the Crypto package. You can test everything is right with:`python -m Crypto.SelfTest`*One must avoid having both PyCrypto and PyCryptodome installed at the same time, as they will interfere with each other.* SelfTest will cost 1 min

2.a library independent of the old PyCrypto. You install it with:`pip install pycryptodomex`
You can test everything is right with:`python -m Cryptodome.SelfTest`In this case, all modules are installed under the Cryptodome package. PyCrypto and PyCryptodome can coexist.

see more:https://www.pycryptodome.org/en/latest/src/installation.html

*If you install PyCrypto without Microsoft Visual C++, you will in big trouble. But you can just install pycryptodome*   
I got this error:

warning: GMP or MPIR library not found; Not building Crypto.PublicKey._fastmath.
    building 'Crypto.Random.OSRNG.winrandom' extension
    error: Microsoft Visual C++ 14.0 is required. Get it with "Microsoft Visual C++ Build Tools": http://landinghub.visualstudio.com/visual-cpp-build-tools

https://stackoverflow.com/questions/11405549/how-do-i-install-pycrypto-on-windows
https://stackoverflow.com/questions/32800336/pycrypto-on-python-3-5  
https://www.pycryptodome.org/en/latest/src/installation.html

# Generating keys

In [43]:
(pubkey, privkey) = rsa.newkeys(512)
print (pubkey)
print (privkey)

PublicKey(10519164108389327680989331008616234297236516678424550755733276376281916813155014277755530108691567124482554158100801509442805895676698443704879180602121301, 65537)
PrivateKey(10519164108389327680989331008616234297236516678424550755733276376281916813155014277755530108691567124482554158100801509442805895676698443704879180602121301, 65537, 5299628614840453810380483257434591513590114098726243448018696604850333844010363876338461682518046032131688083076790560135170425848973724261999215187323329, 6113182022003849817327350375553650095846789869424589878836754638052505984441624009, 1720734646952526676610337162362933351218813745386064377250936326400620589)


Another way to speed up the key generation process is to use multiple processes in parallel to speed up the key generation. Use no more than the number of processes that your machine can run in parallel; a dual-core machine should use poolsize=2; a quad-core hyperthreading machine can run two threads on each core, and thus can use poolsize=8.

In [44]:
(pubkey, privkey) = rsa.newkeys(512, poolsize=multiprocessing.cpu_count())
print (pubkey)
print (privkey)

PublicKey(8227028887144762825046718973385699756471149125334116214488109713716508819760832523501897909634011905925863156736733180799190253477537122910133800942644427, 65537)
PrivateKey(8227028887144762825046718973385699756471149125334116214488109713716508819760832523501897909634011905925863156736733180799190253477537122910133800942644427, 65537, 96032425937497040773314921565528790052953737291615406626537740985903066091149356965262986517966736447278112596377464264842445823729134226917420161480473, 6145596695454138806784876762557918663089765192010765062101803339730184369619648101, 1338686753270068468145374996615870112285184182291111208239443594560975727)


# Encryption and decryption

To encrypt or decrypt a message, use rsa.encrypt() resp. rsa.decrypt().
Let’s say that Alice wants to send a message that only Bob can read.

1.Bob generates a keypair, and gives the public key to Alice. This is done such that Alice knows for sure that the key is really Bob’s (for example by handing over a USB stick that contains the key).

In [45]:
(bob_pub, bob_priv) = rsa.newkeys(512)

2.Alice writes a message, and encodes it in UTF-8. The RSA module only operates on bytes, and not on strings, so this step is necessary.

In [46]:
message = 'hello Bob!'.encode('utf8')

3.Alice encrypts the message using Bob’s public key, and sends the encrypted message.

In [47]:
crypto = rsa.encrypt(message, bob_pub)

4.Bob receives the message, and decrypts it with his private key.

In [48]:
message = rsa.decrypt(crypto, bob_priv)
print(message.decode('utf8'))

hello Bob!


*RSA can only encrypt messages that are smaller than the key. A couple of bytes are lost on random padding, and the rest is available for the message itself. *For example, a 512-bit key can encode a 53-byte message (512 bit = 64 bytes, 11 bytes are used for random padding and other stuff). See Working with big files for information on how to work with larger files.

# Signing and verification

create a detached signature
This hashes the message using SHA-1. Other hash methods are also possible
The RSA module only operates on bytes, and not on strings, so this step is necessary.

In [49]:
(pubkey, privkey) = rsa.newkeys(512)
message = 'Go left at the blue tree'
signature = rsa.sign(message.encode('utf8'), privkey, 'SHA-1')
print (signature)

b"\x06@G?\x9af2[T\x12\xc5b\xd7\xcd\xfa\xfc\x8d\x17\x98\x00'\xf2f\xa5\xb1.\xb8\x1by\xb7R\x1e!\x0f3\xa3\x1ad\xb4,\x9fHM9\xffD\xbc\x90\x0f\x8b\xab\xf7}\x1e/\xd5\xd3\xf8\x02\xb7;\xb5\x1b\xed"


verify the signature

In [50]:
message = 'Go left at the blue tree'
rsa.verify(message.encode('utf8'), signature, pubkey)

True

*Never display the stack trace of a rsa.pkcs1.VerificationError exception. It shows where in the code the exception occurred, and thus leaks information about the key. It’s only a tiny bit of information, but every bit makes cracking the keys easier.*

# Working with big files

RSA can only encrypt messages that are smaller than the key.The most common way to use RSA with larger files uses a block cypher like AES or DES3 to encrypt the file with a random key, then encrypt the random key with RSA. You would send the encrypted file along with the encrypted key to the recipient. The complete flow is:

## Generate a random key

In [51]:
import rsa.randnum
aes_key = rsa.randnum.read_random_bits(16*8) # 16 bytes
print (aes_key)

b'\xbc8|&\xecb\xdb\xd56\x06\xefv\x95\\\xb0\x8e'


## Use that key to encrypt the file with AES.

message.txt : original file  
encrypted.bin : encrypted file  
decrypted.txt: decrypted file

### Encrypt data with AES

In [56]:
from Crypto.Cipher import AES
with open('message.txt','rb') as f:
    data = f.read()
    f.close()
cipher = AES.new(aes_key,AES.MODE_EAX)
ciphertext, tag = cipher.encrypt_and_digest(data)
file_out = open('encrypted.bin','wb')
[ file_out.write(x) for x in (cipher.nonce, tag, ciphertext) ]
print(len(cipher.nonce))

16


## Encrypt the AES key with RSA

In [53]:
(public_rsa_key, privkey) = rsa.newkeys(512)
encrypted_aes_key = rsa.encrypt(aes_key, public_rsa_key)

4.Send the encrypted file together with encrypted_aes_key

5.The recipient now reverses this process to obtain the encrypted file.

In [57]:
from Crypto.Cipher import AES
key = rsa.decrypt(encrypted_aes_key,privkey)
print (key == aes_key)
file_in = open("encrypted.bin", "rb")
nonce, tag, ciphertext = [ file_in.read(x) for x in (16, 16, -1) ]
print(len(nonce))

cipher = AES.new(key, AES.MODE_EAX, nonce)
decrypted_data = cipher.decrypt_and_verify(ciphertext, tag)
with open('decrypted.txt','wb') as f:
    f.write(decrypted_data)
    f.close()

True
16


In [58]:
print(data)

b"The Witcher 3: Wild Hunt[a] is a 2015 action role-playing video game developed and published by CD Projekt. \r\nBased on The Witcher series of fantasy novels by Polish author Andrzej Sapkowski, \r\nit is the sequel to the 2011 video game The Witcher 2: Assassins of Kings and the third installment in The Witcher video game series. \r\nPlayed in an open world with a third-person perspective, players control protagonist Geralt of Rivia. \r\nGeralt, a monster hunter known as a Witcher, is looking for his missing adopted daughter, \r\nwho is on the run from the Wild Hunt: an otherworldly force determined to capture and use her powers. \r\nPlayers battle the game's many dangers with weapons and magic, interact with non-player characters, \r\nand complete main-story and side quests to acquire experience points and gold, \r\nwhich are used to increase Geralt's abilities and purchase equipment. Its central story has several endings, \r\ndetermined by the player's choices at certain points in 

In [59]:
print(decrypted_data)

b"The Witcher 3: Wild Hunt[a] is a 2015 action role-playing video game developed and published by CD Projekt. \r\nBased on The Witcher series of fantasy novels by Polish author Andrzej Sapkowski, \r\nit is the sequel to the 2011 video game The Witcher 2: Assassins of Kings and the third installment in The Witcher video game series. \r\nPlayed in an open world with a third-person perspective, players control protagonist Geralt of Rivia. \r\nGeralt, a monster hunter known as a Witcher, is looking for his missing adopted daughter, \r\nwho is on the run from the Wild Hunt: an otherworldly force determined to capture and use her powers. \r\nPlayers battle the game's many dangers with weapons and magic, interact with non-player characters, \r\nand complete main-story and side quests to acquire experience points and gold, \r\nwhich are used to increase Geralt's abilities and purchase equipment. Its central story has several endings, \r\ndetermined by the player's choices at certain points in 